In [8]:
import polars as pl
pl.Config.set_fmt_str_lengths(50)


file='https://drive.google.com/u/0/uc?id=1pSRvEPmZ8h6a3k6t6_-7hwR-4khiQB81&export=download'

df=pl.read_csv(file)



In [9]:
df.head()

Customer ID,Mobile App - Ease of Use,Mobile App - Ease of Access,Mobile App - Navigation,Mobile App - Likelihood to Recommend,Mobile App - Overall Rating,Online Interface - Ease of Use,Online Interface - Ease of Access,Online Interface - Navigation,Online Interface - Likelihood to Recommend,Online Interface - Overall Rating
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
535084,2,1,5,4,1,4,4,5,2,3
250892,3,5,4,4,2,5,5,2,4,3
544191,5,3,4,4,1,3,3,2,3,1
949343,2,5,4,3,1,1,4,3,5,1
915305,3,1,2,1,1,4,2,4,3,2


In [11]:
import polars as pl

# Assuming df is your dataframe

# Split columns based on platform
mobile_app_cols = [col for col in df.columns if "Mobile App" in col]
online_interface_cols = [col for col in df.columns if "Online Interface" in col]

# Melt the dataframes
mobile_app_melted = df.melt(id_vars=["Customer ID"], value_vars=mobile_app_cols, 
                            variable_name="Question", value_name="Mobile App Rating")
                            
online_interface_melted = df.melt(id_vars=["Customer ID"], value_vars=online_interface_cols, 
                                  variable_name="Question", value_name="Online Interface Rating")

# Rename the questions to remove platform-specific prefix
mobile_app_melted = mobile_app_melted.with_columns(mobile_app_melted["Question"].apply(lambda x: x.replace("Mobile App - ", "")))
online_interface_melted = online_interface_melted.with_columns(online_interface_melted["Question"].apply(lambda x: x.replace("Online Interface - ", "")))

# Join the melted dataframes
reshaped_df = mobile_app_melted.join(online_interface_melted, on=["Customer ID", "Question"], how="inner").select(
    "Customer ID", "Question", "Mobile App Rating", "Online Interface Rating"
)


In [12]:
reshaped_df.filter(pl.col('Customer ID') == 535084)

Customer ID,Question,Mobile App Rating,Online Interface Rating
i64,str,i64,i64
535084,"""Ease of Use""",2,4
535084,"""Ease of Access""",1,4
535084,"""Navigation""",5,5
535084,"""Likelihood to Recommend""",4,2
535084,"""Overall Rating""",1,3
